# __Demo: Creating a Custom Template__



#this example was used for his friend in HR that only wanted a prompt specific to talent acquisition

__Steps to Perform__

Step 1: Set up the Environment

Step 2: Define the Prompt Template

Step 3: Create a Custom Prompt Template Class



### __Step 1: Set up the Environment__

- Import required modules such as StringPromptTemplate from the langchain.prompts library and BaseModel and validator from Pydantic library.



In [1]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator, ValidationError
from typing import List
import openai

### __Step 2: Define the Prompt Template__
- Define a constant string PROMPT that outlines the structure of the prompt to be used for generating book summaries.
- Include placeholders for dynamic content like the book title.

In [2]:
PROMPT = """\
Given the book title, generate a brief summary of the book.
Book Title: {book_title}
Summary:
"""


### __Step 3: Create a Custom Prompt Template Class__

- The custom class BookSummarizerPrompt is created, which does not inherit from __StringPromptTemplate__ as initially described.
- It has a template field of type __StringPromptTemplate__. This class is responsible for integrating the prompt template with dynamic content (the book title) and is built on BaseModel from Pydantic to enable input validation.
- The Config class within __BookSummarizerPrompt__ sets __arbitrary_types_allowed = True__, allowing for the inclusion of custom types like __StringPromptTemplate__ as a field in the Pydantic model.
- A __create_prompt__ method is implemented to dynamically insert the book title into the prompt template.

**Pydantic** is a Python library that helps check and organize data. It ensures the data is correct and in the right format, automatically fixing types if needed. It is useful for validating user input, API requests, and managing settings easily.

In [3]:
# let's first understand how Pydantic work
# Define a Pydantic model
class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    friends: List[str] = []

# # Valid data
data = {
    "id": 123,
    "name": "Alice",
    "email": "alice@example.com",
    "age": 25,
    "friends": ["Raghav", "Afaque", "Ankit"]
}

# # Instantiate the model with valid data
# user = User(**data)
# print(user)

# Invalid data (age should be an integer)
# invalid_data = {
#     "id": 124,
#     "name": "Bob",
#     "email": "bob@example.com",
#     "age": "twenty-five",  # Invalid age
#     "friends": [234, 345]
# }

user = User(**data)
print(user)
# try:
#     invalid_user = User(**invalid_data)
# except ValidationError as e:
#     print(e.json())


id=123 name='Alice' email='alice@example.com' age=25 friends=['Raghav', 'Afaque', 'Ankit']


In [4]:
# Define a Pydantic model
class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    friends: List[str] = []

    class Config:
        strict = True
# # # Valid data
# data = {
#     "id": 123,
#     "name": "Alice",
#     "email": "alice@example.com",
#     "age": 25,
#     "friends": ["Raghav", "Afaque", "Ankit"]
# }

# # # Instantiate the model with valid data
# # user = User(**data)
# # print(user)

# Invalid data (age should be an integer)
invalid_data = {
    "id": 124,
    "name": "Bob",
    "email": "bob@example.com",
    "age": 25,  # Invalid age
    "friends": [234, 345]
    # "friends": ["Raghav", "Afaque", "Ankit"]
}

user = User(**invalid_data)
print(user)
# try:
#     invalid_user = User(**invalid_data)
# except ValidationError as e:
#     print(e.json())


ValidationError: 2 validation errors for User
friends.0
  Input should be a valid string [type=string_type, input_value=234, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
friends.1
  Input should be a valid string [type=string_type, input_value=345, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [ ]:

class StringPromptTemplate:
    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

class BookSummarizerPrompt(BaseModel):
    book_title: str
    template: StringPromptTemplate

    class Config:
        arbitrary_types_allowed = True

    def create_prompt(self) -> str:
        return self.template.format(book_title=self.book_title)

# Define the template
template = StringPromptTemplate(
    """
Given the book title, generate a brief summary of the book.
Book Title: {book_title}
Summary:
"""
)

# Create instance of BookSummarizerPrompt
book_summarizer_prompt = BookSummarizerPrompt(book_title="The Great Gatsby", template=template)

# Generate prompt
formatted_prompt = book_summarizer_prompt.create_prompt()
print("Generated Prompt:")
print(formatted_prompt)


# initialize the OpenAI client using your API key
from google.colab import userdata
import openai

# Fetch API key from Colab secrets vault
api_key = userdata.get("OpenAI")

# Create OpenAI client
client = openai.OpenAI(api_key=api_key)


# Define OpenAI completion function
def get_completion(prompt: str, model="gpt-3.5-turbo") -> str:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            top_p=0.8,
            max_tokens=512
        )

        return response.choices[0].message.content

    except openai.OpenAIError as e:
        return f"OpenAI API Error: {str(e)}"
    except Exception as e:
        return f"Unexpected Error: {str(e)}"

# Get AI-generated summary
response = get_completion(formatted_prompt)
print("\nAI Response:")
print(response)


Generated Prompt:

Given the book title, generate a brief summary of the book.
Book Title: The Great Gatsby
Summary:


AI Response:
"The Great Gatsby" follows the story of Jay Gatsby, a wealthy and mysterious man who throws extravagant parties in the hopes of winning back his lost love, Daisy Buchanan. Set in the 1920s, the novel explores themes of love, wealth, and the American Dream, as Gatsby's obsession with Daisy leads to tragic consequences for all involved. Through the eyes of narrator Nick Carraway, the reader is taken on a journey through the glamorous and ultimately destructive world of the wealthy elite in Jazz Age America.


In [ ]:
#Assignment: dynamically generate custom interview questions from LLMs for a specific job role
#class = StringPromptTemplate
#class = InterviewQuestions (job_role, years_of_experience)


class StringPromptTemplate:
    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

class InterviewQuestionsPrompt(BaseModel):
    job_title: str
    years_of_experience: int # Add years_of_experience field
    template: StringPromptTemplate

    class Config:
        arbitrary_types_allowed = True

    def create_prompt(self) -> str:
        return self.template.format(job_title=self.job_title, years_of_experience=self.years_of_experience) # Pass years_of_experience to format

# Define the template
template = StringPromptTemplate(
    """
Given the job title and years of experience, generate a list of five interview questions.
Job Title: {job_title}
Years of Experience: {years_of_experience}
Interview Questions:
"""
)

# Create instance of InterviewQuestionsPrompt
interview_questions_prompt = InterviewQuestionsPrompt(job_title="Product Owner", years_of_experience=10, template=template)

# Generate prompt
formatted_prompt = interview_questions_prompt.create_prompt()
print("Generated Prompt:")
print(formatted_prompt)

# initialize the OpenAI client using your API key
from google.colab import userdata
import openai

# Fetch API key from Colab secrets vault
api_key = userdata.get("OpenAI")

# Create OpenAI client
client = openai.OpenAI(api_key=api_key)


# Define OpenAI completion function
def get_completion(prompt: str, model="gpt-3.5-turbo") -> str:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            top_p=0.8,
            max_tokens=512
        )

        return response.choices[0].message.content

    except openai.OpenAIError as e:
        return f"OpenAI API Error: {str(e)}"
    except Exception as e:
        return f"Unexpected Error: {str(e)}"

# Get AI-generated summary
response = get_completion(formatted_prompt)
print("\nAI Response:")
print(response)

Generated Prompt:

Given the job title and years of experience, generate a list of five interview questions.
Job Title: Product Owner
Years of Experience: 10
Interview Questions:


AI Response:
1. Can you walk us through a successful product that you have owned from conception to launch? What were the key challenges you faced and how did you overcome them?
2. How do you prioritize features and requirements when developing a new product? Can you provide an example of a time when you had to make tough decisions on what to include or exclude?
3. How do you collaborate with cross-functional teams, such as developers, designers, and stakeholders, to ensure the successful delivery of a product?
4. How do you gather and incorporate feedback from customers and stakeholders throughout the product development process? Can you provide an example of a time when feedback significantly influenced the direction of a product?
5. How do you stay current with industry trends and best practices in produc

In [ ]:
# Define a template class to manage dynamic text
class RecipePromptTemplate: #responsible for defining a template class that can be reused for several recipes
    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs):
        return self.template.format(**kwargs)

# Define a pydantic model for recipe inputs
class RecipeGenerator(BaseModel):
    recipe_name: str
    ingredients: List[str]
    template: RecipePromptTemplate

    def create_prompt(self):
        # Join ingredients into a single string for the template
        ingredients_text = ", ".join(self.ingredients)
        return self.template.format(recipe_name=self.recipe_name, ingredients=ingredients_text)

    class Config:
        arbitrary_types_allowed = True


#template=RecipePromptTemplate ("Write a detailed recipe for {recipe_name} using these ingredients:{ingredients}")
#Generator=RecipeGenerator(recipe_name="hummus", ingredients=['chick peas', 'salt', 'red peppers'])
#print(generator.create_prompt()))

# Define the prompt template
recipe_template = RecipePromptTemplate(
    """
    Generate a detailed recipe for the following dish:
    Recipe Name: {recipe_name}
    Ingredients: {ingredients}
    Instructions:
    """
)

# Create an instance of the recipe generator
recipe_data = {
    "recipe_name": "Pizza",
    "ingredients": ["Refined Flour", "eggs", "parmesan cheese", "Mushrooms", "black pepper"],
    "template": recipe_template,
}

recipe_generator = RecipeGenerator(**recipe_data)

# Generate the prompt
prompt = recipe_generator.create_prompt()
print("Prompt:")
print(prompt)
response = get_completion(recipe_generator.create_prompt())
print("AI Response:")
print(response)

Prompt:

    Generate a detailed recipe for the following dish:
    Recipe Name: Pizza
    Ingredients: Refined Flour, eggs, parmesan cheese, Mushrooms, black pepper
    Instructions:
    
AI Response:
1. Preheat your oven to 450°F (230°C).
2. In a mixing bowl, combine 2 cups of refined flour, 2 eggs, and a pinch of salt. Mix well until a dough forms.
3. On a floured surface, knead the dough for about 5 minutes until it becomes smooth and elastic.
4. Roll out the dough into a round shape, about 12 inches in diameter, and place it on a greased pizza pan.
5. Spread a layer of tomato sauce over the dough, leaving a small border around the edges.
6. Sprinkle a generous amount of grated parmesan cheese over the sauce.
7. Slice some mushrooms and arrange them on top of the cheese.
8. Season the pizza with freshly ground black pepper to taste.
9. Bake the pizza in the preheated oven for 15-20 minutes, or until the crust is golden brown and the cheese is melted and bubbly.
10. Remove the pizza

## __Conclusion__
This demo is designed to dynamically generate book summaries and recipies by inserting book titles and ingredients into a structured prompt template.

It achieves this through a custom implementation involving a template class for formatting and a Pydantic model for data validation and dynamic content integration, ensuring the output is tailored and accurate.

In [15]:
class StringPromptTemplate:
  def __init__(self, template: str):
    self.template = template

  def format(self, **kwargs) -> str:
    return self.template.format(**kwargs)

class UserStoryPrompt(BaseModel):
  persona: str
  action: str
  goal: str
  template: StringPromptTemplate

  class Config:
    arbitrary_types_allowed = True

  def create_prompt(self) -> str:
    return self.template.format(persona=self.persona, action=self.action, goal=self.goal)

#define the template
template = StringPromptTemplate(
    """
    Given the persona, action, and goal, generate a detailed user story in the format of As a..., I want..., so that.... Include 3 acceptance criteria in bullet format and 2 edge cases.
    Persona: {persona}
    Action: {action}
    Goal: {goal}
    User Story:
    """
)

#create instance of userstorypromt
user_story_prompt = UserStoryPrompt (persona='Study Director', action='access reports for studies I manage', goal='view status of studies', template=template)

#generate prompt
formatted_prompt = user_story_prompt.create_prompt()
print("Generated Prompt:")
print(formatted_prompt)

#initialize openAI client using your API key
from google.colab import userdata
import openai

#fetch API key from Colab secrets vault
api_key = userdata.get("OpenAI")

#create OpenAI client
client = openai.OpenAI(api_key=api_key)

#fetch API key from Colab secrets
api_key = userdata.get("OpenAI")

#create OpenAI client
client = openai.OpenAI(api_key=api_key)

#define OpenAI completion function
def get_completion(prompt: str, model='gpt-3.5-turbo') -> str:
  try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        top_p=0.7,
        max_tokens=512
    )

    return response.choices[0].message.content

  except openai.OpenAIError as e:
    return f"OpenAI API Error: {str(e)}"
  except Exception as e:
    return f"Unexpected Error: {str(e)}"

#get AI generated summary
response = get_completion(formatted_prompt)
print("\nAI Response:")
print(response)

Generated Prompt:

    Given the persona, action, and goal, generate a detailed user story in the format of As a..., I want..., so that.... Include 3 acceptance criteria in bullet format and 2 edge cases.
    Persona: Study Director
    Action: access reports for studies I manage
    Goal: view status of studies
    User Story:
    

AI Response:
As a Study Director, I want to access reports for studies I manage so that I can view the status of each study easily.

Acceptance Criteria:
- I can log in to the system using my unique credentials.
- I can navigate to the "Reports" section where all the reports for studies I manage are listed.
- I can click on a specific report to view detailed information about the study's status.

Edge Case 1: 
- If I enter incorrect login credentials, I should receive an error message prompting me to try again.

Edge Case 2:
- If the system is experiencing technical difficulties and I am unable to access the reports, I should receive a notification informi

In [19]:
#use case: use User Story and Acceptance Criteria to generate test cases

class TestCaseTemplate:
  def __init__(self, template: str):
    self.template = template

  def format(self, **kwargs) -> str:
    return self.template.format(**kwargs)

class TestCaseGenerator(BaseModel):
  user_story: str
  acceptance_criteria: List[str]
  template: TestCaseTemplate

  def create_prompt(self) -> str: #join AC together into one string for the template
   acceptance_criteria = '\n'.join(f"- {item}" for item in self.acceptance_criteria)
   return self.template.format(user_story=self.user_story, acceptance_criteria=acceptance_criteria)

  class Config:
    arbitrary_types_allowed = True #means pydantic will allow the custom class I'm defining

template = TestCaseTemplate(
    """
From this User Story: {user_story}

and these Acceptance Criteria:
{acceptance_criteria}

Generate 5 test cases, including 2 negative test cases.

Test Cases:
"""
)

#create an instance of the test case generator
test_case_data = {
    "user_story": "As a Study Director, I want to access reports for studies I manage so that I can view the status of each study easily.",
    "acceptance_criteria":
      ["I can log in to the system using my unique credentials.",
      "I can navigate to the 'Reports' section where all the reports for studies I manage are listed.",
      "I can click on a specific report to view detailed information about the study's status."],
    "template": template

}

test_case_generator = TestCaseGenerator(**test_case_data)

#generate prompt
formatted_prompt = test_case_generator.create_prompt()
print("Generated Prompt:")
print(formatted_prompt)

#OpenAI stuff
from google.colab import userdata
import openai

api_key = userdata.get("OpenAI")

client = openai.OpenAI(api_key=api_key)

#define completion function
def get_completion(prompt: str, model="gpt-4o") -> str:
  try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        top_p=0.7,
        max_tokens=900
    )

    return response.choices[0].message.content

  except openai.OpenAIError as e:
    return f"OpenAI API Error: {str(e)}"
  except Exception as e:
    return f"Unexpected Error: {str(e)}"

response = get_completion(formatted_prompt)
print("\nAI Response:")
print(response)

Generated Prompt:

From this User Story: As a Study Director, I want to access reports for studies I manage so that I can view the status of each study easily.

and these Acceptance Criteria:
- I can log in to the system using my unique credentials.
- I can navigate to the 'Reports' section where all the reports for studies I manage are listed.
- I can click on a specific report to view detailed information about the study's status.

Generate 5 test cases, including 2 negative test cases.

Test Cases:


AI Response:
Certainly! Here are five test cases based on the provided user story and acceptance criteria, including two negative test cases:

### Test Case 1: Successful Login with Valid Credentials
- **Objective:** Verify that the Study Director can log in to the system using valid credentials.
- **Preconditions:** The Study Director has a registered account with valid credentials.
- **Steps:**
  1. Navigate to the login page.
  2. Enter valid username and password.
  3. Click the "Lo